In [1]:
import json
import pandas as pd
import tqdm

In [26]:
with open('ultimate_games_text.json', 'r') as file:
    ultimate_games_text = json.load(file)

corpus = []
titles = []

In [27]:
for game_title, game in ultimate_games_text.items():
    titles.append(game_title)
    corpus.append(game_title + ':\n\n' + '\n\n'.join(list(game.values())))

In [29]:
print(corpus[0])

007 - The World Is Not Enough:

Metacritic Description: Experience the intensity of being the world's top secret agent. Equipped with a full arsenal of of Q-Lab gadgets and weaponry, you must be suave, resourceful, and lethal as you carry out action-packed missions based on the blockbuster movie. Are you cool under pressure? Deadly when necessary? Of course you are -- you're Bond...James Bond.


GiantBomb:
Intro: The World Is Not Enough, a video game adaption of the actual hollywood movie, follows James Bond on his secret mission both to unravel the mysterious intentions of a femmè fatale called Elektra King and to uncover the plans of his vicious counterpart Renard.
Description: Overview Coming off the failure that was  Tomorrow Never Dies , publisher  EA  wanted to return to  James Bond's  roots as a  first-person  shooter. The game, based on the film of the same name, had a different developer for each of the three platforms that it was released on. The  Nintendo 64  version was dev

In [5]:
from sentence_transformers import SentenceTransformer, util
import torch

In [30]:
# Step 1: Load the pre-trained model (this can be any model that works well for semantic search)
model = SentenceTransformer('all-MiniLM-L6-v2')

c:\Users\SCO25143\Downloads\PyProject\openenv\Lib\site-packages\huggingface_hub\file_download.py:139: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\SCO25143\.cache\huggingface\hub\models--sentence-transformers--all-MiniLM-L6-v2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


In [ ]:
# Step 3: Encode the documents
document_embeddings = model.encode(corpus, convert_to_tensor=True)

# Step 5: Use util.pytorch_cos_sim to get pairwise cosine similarities
cosine_scores = util.pytorch_cos_sim(document_embeddings, document_embeddings)

In [ ]:
# Step 4: Find k most similar documents to each document using cosine similarity
num_similar_documents = 20
# Step 6: For each document, get the 20 most similar documents (excluding the document itself)
for idx, scores in enumerate(cosine_scores):
    # Sort the cosine scores in descending order and exclude the first one (which is the document itself)
    top_results = torch.topk(scores, num_similar_documents + 1)  # +1 to include itself, we'll slice later
    most_similar_idx = top_results.indices[1:]  # Exclude the first one (itself)
    
    print(titles[idx], '#', '; '.join([titles[v] for v in most_similar_idx]))
    ultimate_games_text[titles[idx]]['Similar Games'] = [titles[v] for v in most_similar_idx]

In [49]:
ultimate_games_text['Hollow Knight']

{'my-steam': "My Steam:\nSummary: About This Game Hollow Knight Expands with Free ContentGodmaster - Take your place amongst the Gods. New Characters and Quest. New Boss Fights. Available Now!Lifeblood - A Kingdom Upgraded! New Boss. Upgraded Bosses. Tweaks and Refinements across the whole game.The Grimm Troupe - Light the Nightmare Lantern. Summon the Troupe. New Major Quest. New Boss Fights. New Charms. New Enemies. New Friends.Hidden Dreams - Mighty new foes emerge! New Boss fights. New Upgrades. New Music.Brave the Depths of a Forgotten KingdomBeneath the fading town of Dirtmouth sleeps an ancient, ruined kingdom. Many are drawn below the surface, searching for riches, or glory, or answers to old secrets.Hollow Knight is a classically styled 2D action adventure across a vast interconnected world. Explore twisting caverns, ancient cities and deadly wastes; battle tainted creatures and befriend bizarre bugs; and solve ancient mysteries at the kingdom's heart.Game FeaturesClassic side

In [50]:
with open('ultimate_games_text.json', 'w') as file:
    json.dump(ultimate_games_text, file)